In [ ]:
rm(list = ls())

library(reshape2)
library(dplyr)
library(xgboost)
library(ggplot2)
data = read.csv("data.csv")
str(data)

trade = data %>% dplyr::select(acc_id, wk, from_accessory, from_costume, from_gem, from_grocery, from_money, from_weapon,
                        to_accessory, to_costume, to_gem, to_grocery, to_money, to_weapon)
trade_nonzero = subset(trade,  ! apply(trade[, -c(1, 2)], 1, sum) == 0 )

In [ ]:
#######################################
# Scree plot : 최적의 factor수 구하기 #
#######################################
cor <- cor(trade_nonzero[, -c(1, 2)]);cor
pca.fit <- prcomp(trade_nonzero[, -c(1, 2)],center=T,scale=T)   # standardizing 필요함
summary(pca.fit)
variance <- pca.fit$sdev^2
plot(x=c(1:length(variance)), y=eigen(cor)$values, type="b", main="Scree plot",xlab="",ylab="variance")
segments(0,1,23,1,col="red")

############
# EFA 실시 #
############
efa.fit = factanal(trade_nonzero[, -c(1, 2)], factors = 4, rotation = "varimax", scores = "Bartlett");efa.fit
efa.fit

###################
# trade_label부여 #
###################
efa.scores = efa.fit$scores
efa.loadings = efa.fit$loadings
trade_label = apply(efa.scores, 1, which.max)
trade_fac = cbind(trade_nonzero[, c(1, 2)], trade_label)
head(trade_fac)
nrow(trade_fac)



##########################
# loadings matrix 시각화 #
##########################
efa.loadings <- efa.loadings[,1:4]
id <- dimnames(efa.loadings)[[1]]
measure <- dimnames(efa.loadings)[[2]]
efa.loadings.m <- melt(efa.loadings, id=id, measure=measure)
dimnames(efa.loadings.m)[[2]] <- c("Variable", "Factor", "Loading")
efa.loadings.m

ggplot(efa.loadings.m, aes(Variable, abs(Loading), fill=Loading)) + 
  facet_wrap(~ Factor, nrow=1) + #place the factors in separate facets
  geom_bar(stat="identity") + #make the bars
  coord_flip() + #flip the axes so the test names can be horizontal  
  #define the fill color gradient: blue=positive, red=negative
  scale_fill_gradient2(name = "Loading", 
                       high = "blue", mid = "white", low = "red", 
                       midpoint=0, guide=F) +
  ylab("Loading Strength") + #improve y-axis label
  theme_bw(base_size=10) +
  theme(axis.text.y = element_text(size=15))#use a black-and0white theme with set font size


